'''
This is the definition of the notebook
'''

In [ ]:
%reload_ext autoreload
%autoreload 2

## Importing Dataset and Libraries

In [ ]:
'''
Required libraries. Some of them may not be required.
'''

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib
import pandas as pd #req.
import numpy as np
import nltk
import sys
import xlsxwriter
import re
import string
from sqlalchemy import create_engine
import time
import logging
import string
import emoji
import seaborn as sns
from collections import Counter, defaultdict
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords, wordnet # req.
from sklearn import model_selection, naive_bayes, svm
from sklearn.decomposition import LatentDirichletAllocation, NMF
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from gensim.models import Word2Vec
from zemberek import ( #req.
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
import gensim
import zeyrek
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import confusion_matrix, classification_report
from simpletransformers.classification import ClassificationModel
from sklearn.naive_bayes import BernoulliNB
from sklearn import preprocessing
from gensim.test.utils import common_texts
from sklearn.metrics.pairwise import linear_kernel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import os
from gensim.corpora import Dictionary
import gensim
from gensim.matutils import cossim
import tensorflow as tf
import tensorflow_hub as hub
pd.options.mode.chained_assignment = None  # default='warn'

#### Loading the data

In [ ]:
data=pd.read_csv('tvseries_full.csv', delimiter=";", encoding='utf-8')

In [ ]:
display(data.sample(20))

In [ ]:
data.Lab.value_counts()

In [ ]:
df = data[data.Lab != 'Junk']

df.Lab.replace("Neutral-Negative", 
           "Neutral", 
           inplace=True)

In [ ]:
df.Lab.value_counts()

### Preprocessing

##### Removing Mentions, Punctuations, HTMLs, Hyperlinks and Hashtags

In [ ]:
df['Content'] = df['Content'].replace('@[A-Za-z0-9]+', '', regex=True).replace('@[A-Za-z0-9]+', '', regex=True)
df['Content']  = df['Content'] .replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
df['Content']  = df['Content'] .replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
df['Content']  = df['Content'] .replace('#[A-Za-z0-9ğüşöçıİĞÜŞÖÇ]+', 
                                  '', regex=True).replace('#[A-Za-z0-9ğüşöçıİĞÜŞÖÇ]+', '', regex=True)
df['Content'] = df['Content'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

#repeated letters and spaces
ss=df['Content'] .copy()
new_st = []
i = 0
for k in ss:
    new_ss = re.sub(r'([A-Za-z0-9ğüşöçıİĞÜŞÖÇ])\1+', r'\1',k)
    new_st.append(new_ss)
    i += 1

df['Content'] = new_st

df['Content']  = df['Content'].str.replace('[^\w\s]','')

# tokenization

logger = logging.getLogger(__name__)

morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)
extractor = TurkishSentenceExtractor()

table = str.maketrans('', '', string.punctuation)
ss2 = df['Content'] .copy()
new_st2 = []
for k in ss2:
    words = k.split()
    stripped = [w.translate(table) for w in words]
    new_st2.append(stripped)
    
df['tokenized'] = new_st2

# lowercase

df['lower'] = df['tokenized'].apply(
    lambda x: [word.lower() for word in x])


#stopwords

stops = set(stopwords.words('turkish'))

df['stopwords_removed'] = df['lower'].apply(
    lambda x: [word for word in x if word not in stops])


# Applying part of speech tags.

df['pos_tags'] = df['stopwords_removed'].apply(nltk.tag.pos_tag)

# Converting part of speeches to wordnet format.

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


df['wordnet_pos'] = df['pos_tags'].apply(
    lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

#  normalization

def normalize_long_text(text):
    normalized_sentences = [normalizer.normalize(word) for word in text]
    normalized_text = " ".join(normalized_sentences)
    return normalized_text

sentences = df['tokenized'].copy()
new_sent = []
start = time.time()

for token in sentences:   
    if token.count('') > 0:
        token = list(filter(('').__ne__, token))
    new_token = normalize_long_text(token)
    new_sent.append(new_token)

logger.info(f"Sentences normalized in: {time.time() - start} s")

splitted_words = []
for sent in new_sent:
    words = sent.split()
    splitted_words.append(words)
    
for token in splitted_words:
    j = 0
    for word in token:
        new_word = word.replace('"', '').replace("’", '').replace("'", '').replace("”", '')
        token[j] = new_word
        j += 1
        
        
# Zeyrek for lemmatization

analyzer = zeyrek.MorphAnalyzer()
lem_sent = []
for sent in splitted_words:
    normalized_sent = []
    for word in sent:
        if word == '':
            continue
        else:
            lem_word = analyzer.lemmatize(word)
            normalized_sent.append(lem_word[0][1][0])
    lem_sent.append(normalized_sent)
    
x = lem_sent.copy()
for sent in x:
    i = 0
    for token in sent:
        sent[i] = token.lower()
        i += 1
lem_sent = x


lem_sent = list(filter(('').__ne__, lem_sent))
df['lemmatized'] = lem_sent
df['lemma_str'] = [' '.join(map(str, l)) for l in df['lemmatized']]

display(df.head(20))

### Label Encoding

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(df.Lab)
df['categorical_label'] = le.transform(df.Lab)

In [ ]:
X = []
Y = []
for idx in df.index:
    X.append(df['lemmatized'][idx])
    Y.append(df['categorical_label'][idx])
# Train-Test splitting

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
labels = ['Negative','Neutral', 'Positive']  # For further use...

In [ ]:


# This utility function will be used to evaluate the other models also.
def show_performance_data(Y_test, Y_pred, model_name):
    print(classification_report(Y_test, Y_pred, target_names=labels))
    tmp_result = classification_report(Y_test, Y_pred, target_names=labels, output_dict=True)
    cm1 = confusion_matrix(Y_test, Y_pred)
    df_cm = pd.DataFrame(cm1, index = [i for i in labels], columns = [i for i in labels])
    plt.figure(figsize = (7,5))
    sns.heatmap(df_cm, annot=True,cmap='gist_earth_r', fmt='g')
    plt.savefig('confusion_mrtx_'+model_name+'.png',bbox_inches = 'tight')
    return tmp_result


In [ ]:

topic = 20
Lda = gensim.models.ldamodel.LdaModel
dictionary = Dictionary(X_train)
train_corpus = [dictionary.doc2bow(doc) for doc in X_train]
model = Lda(corpus=train_corpus, id2word=dictionary, num_topics=topic)
test_corpus = [dictionary.doc2bow(doc) for doc in X_test]
train_x_topics, test_x_topics = [], []
# Convert text data into topic vectors
for t in train_corpus:
    train_x_topics.append(model[t])
for t in test_corpus:
    test_x_topics.append(model[t])
# Prediction
Y_pred = []
for i in range(len(test_x_topics)):
    tst = test_x_topics[i]
    sim = [cossim(tst, tr_t) for tr_t in train_x_topics]
    idx = np.array(sim).argsort()[-1]
    Y_pred.append(Y_train[idx])

In [ ]:
result_lda = show_performance_data(Y_test, Y_pred, 'lda')

In [ ]:

vec = 100
tagged_corpus = [TaggedDocument(d, [i]) for i, d in enumerate(X_train)]
model = Doc2Vec(tagged_corpus, vector_size=vec, window=3, dm=1, min_count=1, workers=4)
Y_pred = []
for a in X_test:
    test_doc_vector = model.infer_vector(a)
    sims = model.docvecs.most_similar(positive = [test_doc_vector])
    Y_pred.append(Y_train[sims[0][0]])
    
result_doc2vec = show_performance_data(Y_test, Y_pred,'doc2vec')

In [ ]:

# Loading USE encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
use_model = hub.load(module_url)
print ("module %s loaded" % module_url)
def get_use_vector(doc):
    tmp = use_model(doc)
    return np.mean(tmp, axis=0)
# Convert text data into vector
train_vectors = []
for doc in X_train:
    try:
        train_vectors.append(get_use_vector(doc))
    except Exception:
        print('error...')
test_vectors = []
for doc in X_test:
    try:
        test_vectors.append(get_use_vector(doc))
    except Exception:
        print('error...')

In [ ]:
Y_pred = []
cosine_similarities = linear_kernel(train_vectors, test_vectors)
for cs in cosine_similarities.T:
    idx = cs.argsort()[-1]
    Y_pred.append(Y_train[idx])
result_use = show_performance_data(Y_test, Y_pred, 'use')

In [ ]:

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(train_vectors, Y_train)
Y_pred = neigh.predict(test_vectors)
result_knn = show_performance_data(Y_test, Y_pred, 'knn')

In [ ]:

clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(train_vectors, Y_train)
Y_pred = clf.predict(test_vectors)
result_rnmdfst = show_performance_data(Y_test, Y_pred, 'rndmfst')

In [ ]:

clf = GradientBoostingClassifier(n_estimators=15, learning_rate=1.0,max_depth=1, random_state=0).fit(train_vectors, Y_train)
Y_pred = clf.predict(test_vectors)
result_grdbst = show_performance_data(Y_test, Y_pred, 'grdbst')

In [ ]:
N = 3
results = {'Doc2Vec':result_doc2vec,  'KNN':result_knn, 'USE':result_use, 'Random Forest':result_rnmdfst, 'Gredient Boost':result_grdbst, 'LDA':result_lda}
ind = np.arange(N) 
width = 0.1
p = []
clr = ['b','g','r','c','m','k','y']
i=0
for a,b in results.items():
    tmp = [b['Negative']['precision'], b['Neutral']['precision'], b['Positive']['precision']] # replace 'precision' with 'recall' or 'f1-score' for the next two plots.
    print(tmp)
    p.append(plt.bar(ind+width*i, tmp, width, color = clr[i]))
    i+=1
plt.xlabel("Sentiments")
plt.ylabel('Precision')
plt.title("Precison comparison of all the models with respect to sentiments")
plt.xticks(ind+width,labels)
plt.legend( tuple(p), tuple(results.keys()), loc='upper center', ncol=2)
plt.show()

In [ ]:

N = 3
ind = np.arange(N) 
width = 0.1
  
r = []
clr = ['b','g','r','c','m','k','y']
i=0
for a,b in results.items():
    tmp = [b['Negative']['recall'], b['Neutral']['recall'], b['Positive']['recall']]
    print(tmp)
    r.append(plt.bar(ind+width*i, tmp, width, color = clr[i]))
    i+=1


plt.xlabel("Sentiments")
plt.ylabel('Recall')
plt.title("Recall comparison of all the models with respect to sentiments")
  
plt.xticks(ind+width,labels)
plt.legend( tuple(r), tuple(results.keys()), loc='upper center', ncol=2)
#plt.show()
plt.savefig('all_recall.png',bbox_inches = 'tight')

In [ ]:
N = 3
ind = np.arange(N) 
width = 0.1
  
f = []
clr = ['b','g','r','c','m','k','y']
i=0
for a,b in results.items():
    tmp = [b['Negative']['f1-score'], b['Neutral']['f1-score'], b['Positive']['f1-score']]
    print(tmp)
    f.append(plt.bar(ind+width*i, tmp, width, color = clr[i]))
    i+=1


plt.xlabel("Sentiments")
plt.ylabel('F1-score')
plt.title("F score comparison of all the models with respect to sentiments")
  
plt.xticks(ind+width,labels)
plt.legend( tuple(f), tuple(results.keys()), loc='upper center', ncol=2)
#plt.show()
plt.savefig('all_f_score.png',bbox_inches = 'tight')

In [ ]:
N = 3
ind = np.arange(N) 
width = 0.1
f = []
clr = ['b','g','r','c','m','k','y']
i=0
for a,b in results.items():
    tmp = [b['weighted avg']['precision'], b['weighted avg']['recall'], b['weighted avg']['f1-score']]
    print(tmp)
    f.append(plt.bar(ind+width*i, tmp, width, color = clr[i]))
    i+=1
plt.xlabel("Metrics")
plt.ylabel('score')
plt.title("Over all weighted scores")
  
plt.xticks(ind+width,['Precision', 'Recall', 'F1-score'])
plt.legend( tuple(f), tuple(results.keys()), loc='upper center', ncol=7)
plt.show()
plt.savefig('all_.png',bbox_inches = 'tight')